In [1]:
!nvidia-smi

Wed Dec 21 08:33:33 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  Off  | 00000000:3B:00.0 Off |                    0 |
| N/A   67C    P0   153W / 250W |   5691MiB / 32510MiB |     88%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import pandas as pd
import torch
from torch.utils.data import Dataset, random_split
from transformers import GPT2Tokenizer, TrainingArguments, Trainer, GPT2LMHeadModel

import transformers
from torch.utils.data import DataLoader, TensorDataset, random_split, RandomSampler, Dataset
import pandas as pd
import numpy as np

import torch.nn.functional as F
import torch

import random
import re
import datetime

import logging
import os
import sys

import datasets
from IPython.display import display, HTML
import random
import nltk  # Here to have a nice missing dependency error message early on
import numpy as np
from datasets import load_dataset, load_metric

import transformers
from filelock import FileLock
from transformers import (
    AutoConfig,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    HfArgumentParser,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    set_seed,
)
from transformers.file_utils import is_offline_mode
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version
from transformers.utils.versions import require_version

nltk.download('punkt')

import wandb
wandb.login()

[nltk_data] Downloading package punkt to
[nltk_data]     /home/rishabh19089/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: rg089. Use `wandb login --relogin` to force relogin


True

In [3]:
# data_sizes = [25, 100, 500]
# ep_sizes = {25: 150, 100: 50, 500: 15}

# for size in data_sizes:
#     names = ['normal', 'explicit', 'prompt']
#     if size > 200: names = names[:1]
#     for name in names:
#         TRAIN_PARAMS = f'{name}_{size}'
#         TRAIN_FILE_PATH = f"data/train_{TRAIN_PARAMS}.csv"
#         VAL_FILE_PATH = f"data/test_{TRAIN_PARAMS}.csv"
#         BASE_PATH = "models/"

#         SAVE_PATH = BASE_PATH + f"gpt_{TRAIN_PARAMS}/"
#         LOGGING_PATH = BASE_PATH + f"gpt_{TRAIN_PARAMS}/"
#         SAVE_MODEL_PATH = BASE_PATH + f"gpt_{TRAIN_PARAMS}_model/"
#         TRAIN_FILE_PATH, SAVE_MODEL_PATH
#         device = "cuda" if torch.cuda.is_available() else "cpu"
#         MODEL_NAME = 'gpt2'
#         tokenizer = GPT2Tokenizer.from_pretrained(MODEL_NAME)
#         model = GPT2LMHeadModel.from_pretrained(MODEL_NAME).to(device)
#         max_input_length = 512
#         max_target_length = 512
#         tokenizer.pad_token = tokenizer.eos_token
#         input_col = "history"
#         train_qns = pd.read_csv(TRAIN_FILE_PATH).loc[:, input_col]
#         val_qns = pd.read_csv(VAL_FILE_PATH).loc[:, input_col]
#         class HistoryDataset(Dataset):
#             def __init__(self, txt_list, tokenizer, max_length):
#                 self.input_ids = []
#                 self.attn_masks = []
#                 self.labels = []
#                 self.action_tokens = [257, 275, 269, 288, 304, 277, 308, 289, 1312, 474]
#                 for txt in txt_list:
#                     encodings_dict = tokenizer(txt, padding="max_length", max_length=max_input_length, truncation=True, return_tensors='pt')
#                     self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
#                     self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))
#                     input_ids = encodings_dict['input_ids']
#                     labels = [[l if l in self.action_tokens else -100 for l in input_id] for input_id in input_ids]
#                     self.labels.append(torch.tensor(labels))

#             def __len__(self):
#                 return len(self.input_ids)

#             def __getitem__(self, idx):
#                 return self.input_ids[idx], self.attn_masks[idx], self.labels[idx]
#         train_dataset = HistoryDataset(train_qns, tokenizer, max_length=max_input_length)
#         val_dataset = HistoryDataset(val_qns, tokenizer, max_length=max_input_length)
#         epochs = ep_sizes[size]
#         lr = 8e-5
#         batch_size = 16
#         training_args = TrainingArguments(
#             output_dir=SAVE_PATH,
#             learning_rate=lr,
#             do_train = True,
#             do_eval = True,
#             evaluation_strategy="steps",
#             per_device_train_batch_size=batch_size,
#             per_device_eval_batch_size=batch_size,
#             weight_decay=0.01,
#             save_total_limit=1,
#             load_best_model_at_end=True,
#             num_train_epochs=epochs,
#             # fp16=True,
#             logging_dir=LOGGING_PATH,
#             logging_steps=50,
#             save_steps=100,
#             report_to = "wandb",
#             # hub_model_id='rg089/gpt2_mwp'
#             )
#         wandb_run = wandb.init(
#             project="rl_project_ad",
#             config={
#                 "per_device_train_batch_size": batch_size,
#                 "learning_rate": lr})

#         now = datetime.datetime.now()
#         current_time = now.strftime("%d.%b.%Y-%-I:%M:%S%p")

#         run_name = SAVE_PATH.rstrip("/").split("/")[-1] + "-" + current_time 
#         wandb_run.name = run_name
#         print(run_name)
#         trainer = Trainer(
#                     model=model,  
#                     args=training_args, 
#                     train_dataset=train_dataset, 
#                     eval_dataset=val_dataset, 
#                     data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
#                                                                     'attention_mask': torch.stack([f[1] for f in data]),
#                                                                     'labels': torch.stack([f[2] for f in data])})

#         trainer.evaluate()
#         trainer.train()
#         import math
#         eval_results = trainer.evaluate()
#         print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")
#         eval_results
#         trainer.save_model(SAVE_MODEL_PATH)
#         wandb_run.finish()
#         del model, tokenizer, trainer, train_dataset, val_dataset
#         torch.cuda.empty_cache()


In [4]:
data_sizes = [2000]
ep_sizes = {25: 150, 100: 50, 500: 15, 2000:10}

for size in data_sizes:
    names = ['normal', 'explicit', 'prompt']
    if size > 200: names = names[:1]
    for name in names:
        TRAIN_PARAMS = f'{name}_{size}'
        TRAIN_FILE_PATH = f"data/train_{TRAIN_PARAMS}.csv"
        VAL_FILE_PATH = f"data/test_{TRAIN_PARAMS}.csv"
        BASE_PATH = "models/"

        SAVE_PATH = BASE_PATH + f"gpt_{TRAIN_PARAMS}/"
        LOGGING_PATH = BASE_PATH + f"gpt_{TRAIN_PARAMS}/"
        SAVE_MODEL_PATH = BASE_PATH + f"gpt_{TRAIN_PARAMS}_model/"
        TRAIN_FILE_PATH, SAVE_MODEL_PATH
        device = "cuda" if torch.cuda.is_available() else "cpu"
        MODEL_NAME = 'gpt2'
        tokenizer = GPT2Tokenizer.from_pretrained(MODEL_NAME)
        model = GPT2LMHeadModel.from_pretrained(MODEL_NAME).to(device)
        max_input_length = 512
        max_target_length = 512
        tokenizer.pad_token = tokenizer.eos_token
        input_col = "history"
        train_qns = pd.read_csv(TRAIN_FILE_PATH).loc[:, input_col]
        val_qns = pd.read_csv(VAL_FILE_PATH).loc[:, input_col]
        class HistoryDataset(Dataset):
            def __init__(self, txt_list, tokenizer, max_length):
                self.input_ids = []
                self.attn_masks = []
                self.labels = []
                self.action_tokens = [257, 275, 269, 288, 304, 277, 308, 289, 1312, 474]
                for txt in txt_list:
                    encodings_dict = tokenizer(txt, padding="max_length", max_length=max_input_length, truncation=True, return_tensors='pt')
                    self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
                    self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))
                    input_ids = encodings_dict['input_ids']
                    labels = [[l if l in self.action_tokens else -100 for l in input_id] for input_id in input_ids]
                    self.labels.append(torch.tensor(labels))

            def __len__(self):
                return len(self.input_ids)

            def __getitem__(self, idx):
                return self.input_ids[idx], self.attn_masks[idx], self.labels[idx]
        train_dataset = HistoryDataset(train_qns, tokenizer, max_length=max_input_length)
        val_dataset = HistoryDataset(val_qns, tokenizer, max_length=max_input_length)
        epochs = ep_sizes[size]
        lr = 8e-5
        batch_size = 16
        training_args = TrainingArguments(
            output_dir=SAVE_PATH,
            learning_rate=lr,
            do_train = True,
            do_eval = True,
            evaluation_strategy="steps",
            per_device_train_batch_size=batch_size,
            per_device_eval_batch_size=batch_size,
            weight_decay=0.01,
            save_total_limit=1,
            load_best_model_at_end=False,
            num_train_epochs=epochs,
            # fp16=True,
            logging_dir=LOGGING_PATH,
            logging_steps=50,
            save_steps=100,
            report_to = "wandb",
            # hub_model_id='rg089/gpt2_mwp'
            )
        wandb_run = wandb.init(
            project="rl_project_ad",
            config={
                "per_device_train_batch_size": batch_size,
                "learning_rate": lr})

        now = datetime.datetime.now()
        current_time = now.strftime("%d.%b.%Y-%-I:%M:%S%p")

        run_name = SAVE_PATH.rstrip("/").split("/")[-1] + "-" + current_time 
        wandb_run.name = run_name
        print(run_name)
        trainer = Trainer(
                    model=model,  
                    args=training_args, 
                    train_dataset=train_dataset, 
                    eval_dataset=val_dataset, 
                    data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
                                                                    'attention_mask': torch.stack([f[1] for f in data]),
                                                                    'labels': torch.stack([f[2] for f in data])})

        trainer.evaluate()
        trainer.train()
        import math
        eval_results = trainer.evaluate()
        print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")
        eval_results
        trainer.save_model(SAVE_MODEL_PATH)
        wandb_run.finish()
        del model, tokenizer, trainer, train_dataset, val_dataset
        torch.cuda.empty_cache()


/home/rishabh19089/CounterSpeech/cs/lib/python3.6/site-packages/ipykernel_launcher.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/home/rishabh19089/CounterSpeech/cs/lib/python3.6/site-packages/ipykernel_launcher.py:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


***** Running Evaluation *****
  Num examples = 2000
  Batch size = 16


gpt_normal_2000-21.Dec.2022-8:35:09AM


Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
/home/rishabh19089/CounterSpeech/cs/lib/python3.6/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2000
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1250


Step,Training Loss,Validation Loss
50,0.891000,3.154609
100,0.444600,2.690629
150,0.324800,3.080282
200,0.240400,3.904620
250,0.182300,4.371871
300,0.138800,4.526446
350,0.115400,4.299789
400,0.091300,5.026977
450,0.078300,6.056256
500,0.065200,5.825807


***** Running Evaluation *****
  Num examples = 2000
  Batch size = 16
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 16
Saving model checkpoint to models/gpt_normal_2000/checkpoint-100
Configuration saved in models/gpt_normal_2000/checkpoint-100/config.json
Model weights saved in models/gpt_normal_2000/checkpoint-100/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 16
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 16
Saving model checkpoint to models/gpt_normal_2000/checkpoint-200
Configuration saved in models/gpt_normal_2000/checkpoint-200/config.json
Model weights saved in models/gpt_normal_2000/checkpoint-200/pytorch_model.bin
Deleting older checkpoint [models/gpt_normal_2000/checkpoint-100] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 16
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 16
Saving model checkpoint to models/gpt_normal_

Saving model checkpoint to models/gpt_normal_2000_model/
Configuration saved in models/gpt_normal_2000_model/config.json


Perplexity: 944.99


Model weights saved in models/gpt_normal_2000_model/pytorch_model.bin


eval/loss,▁▃▂▃▄▄▅▄▅▇▆▇▇█▇▇█▇██▇██████
eval/runtime,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▇▁▁█▅█
eval/samples_per_second,█████████████████████▂██▁▃▁
eval/steps_per_second,█████████████████████▂██▁▃▁
train/epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
train/learning_rate,██▇▇▇▇▆▆▆▅▅▅▅▄▄▄▃▃▃▂▂▂▂▁▁
train/loss,█▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
